In [2]:
import pandas as pd

In [3]:
movies_df = pd.read_csv(r'E:\Projects\Movie_Recommendation\artifacts\movies.csv')
ratings_df = pd.read_csv(r'E:\Projects\Movie_Recommendation\artifacts\ratings.csv')

In [4]:
movies_info = movies_df.merge(ratings_df, on='movieId')

In [5]:
movies_info

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,2,3.5,1141415820
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3,4.0,1439472215
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4,3.0,1573944252
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,858625949
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8,4.0,890492517
...,...,...,...,...,...,...
25000090,209157,We (2018),Drama,119571,1.5,1574280748
25000091,209159,Window of the Soul (2001),Documentary,115835,3.0,1574280985
25000092,209163,Bad Poems (2018),Comedy|Drama,6964,4.5,1574284913
25000093,209169,A Girl Thing (2001),(no genres listed),119571,3.0,1574291826


In [6]:
user_rating_threshold = 500
active_users = movies_info.groupby('userId')['rating'].count() > user_rating_threshold


In [7]:
active_users_indices = active_users[active_users == True].index

In [8]:
active_users

userId
1         False
2         False
3          True
4         False
5         False
          ...  
162537    False
162538    False
162539    False
162540    False
162541    False
Name: rating, Length: 162541, dtype: bool

In [9]:
active_users_indices

Index([     3,     12,     72,     80,    120,    166,    171,    175,    181,
          187,
       ...
       162386, 162387, 162394, 162445, 162481, 162484, 162495, 162508, 162516,
       162519],
      dtype='int64', name='userId', length=9663)

In [10]:
movies_watched_by_active_users = movies_info[movies_info['userId'].isin(active_users_indices)]
movie_rating_threshold = 500
popular_movies = movies_watched_by_active_users.groupby('movieId')['rating'].count() > movie_rating_threshold
popular_movie_indices = popular_movies[popular_movies == True].index

final_list_movies = movies_watched_by_active_users[movies_watched_by_active_users['movieId'].isin(popular_movie_indices)]
final_list_movies = final_list_movies.drop_duplicates()

In [11]:
final_list_movies

,movieId,title,genres,userId,rating,timestamp
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3,4.0,1439472215
6,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,12,4.0,1167582601
35,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,120,5.0,956264593
47,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,166,4.0,942014736
49,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,171,4.5,1074594380
...,...,...,...,...,...,...
24961053,195159,Spider-Man: Into the Spider-Verse (2018),Action|Adventure|Animation|Sci-Fi,161675,3.0,1565376240
24961059,195159,Spider-Man: Into the Spider-Verse (2018),Action|Adventure|Animation|Sci-Fi,161919,4.5,1547069919
24961070,195159,Spider-Man: Into the Spider-Verse (2018),Action|Adventure|Animation|Sci-Fi,162257,4.5,1550942623
24961071,195159,Spider-Man: Into the Spider-Verse (2018),Action|Adventure|Animation|Sci-Fi,162271,3.0,1566338203


In [12]:
final_list_movies['movieId'].nunique()

3950

In [13]:
final_movies_info = final_list_movies[['movieId','title']].drop_duplicates()
user_item = final_list_movies.pivot_table(index='title', columns='userId', values='rating', fill_value=0)


In [14]:
user_item.shape

(3950, 9663)